In [103]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from collections import Counter

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve, train_test_split, KFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import scale
from sklearn.preprocessing import OneHotEncoder
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
import warnings
import mltools as ml
warnings.filterwarnings('ignore')
min_max=MinMaxScaler()

In [133]:
X=pd.read_csv('X_train.csv')
y=pd.read_csv('Y_train.csv')
X_train, X_test, Y_train, Y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)

le=LabelEncoder()
for col in X_test.columns.values:
    if X_test[col].dtypes=='object':
        data=X_train[col].append(X_test[col])
        le.fit(data.values)
        X_train[col]=le.transform(X_train[col])
        X_test[col]=le.transform(X_test[col])

# X_train[X_train.dtypes[(X_train.dtypes=="float64")|(X_train.dtypes=="int64")]
#                         .index.values].hist(figsize=[20,20], color='brown', histtype='step')
        
# X_train.head()

In [134]:
X=pd.read_csv('X_train.csv')
y=pd.read_csv('Y_train.csv')
kaggleTest =pd.read_csv('X_test.csv')

X_traink, X_testk, Y_traink, Y_testk = train_test_split(
    X, y, test_size=0.0, random_state=42)


for col in X_testk.columns.values:
    if X_testk[col].dtypes=='object':
        data=X_traink[col].append(X_testk[col])
        le.fit(data.values)
        X_traink[col]=le.transform(X_traink[col])
        X_testk[col]=le.transform(X_testk[col])

le=LabelEncoder()
for col in kaggleTest.columns.values:
    if kaggleTest[col].dtypes=='object':
        data=kaggleTest[col].append(X_testk[col])
        le.fit(data.values)
        kaggleTest[col]=le.transform(kaggleTest[col])


In [141]:
clf1 = LogisticRegression(penalty='l2',C=.01)
clf2 = RandomForestClassifier(max_depth=11,min_samples_leaf=7,max_features=13,n_estimators=850,bootstrap =True)
clf3 = AdaBoostClassifier(n_estimators=950, learning_rate=1.1)
clf4 = DecisionTreeClassifier(max_depth=9,min_samples_leaf=4)
clf5 = KNeighborsClassifier(n_neighbors=21)
clf6 = GradientBoostingClassifier(n_estimators=40, learning_rate=0.8,max_depth=3, min_samples_leaf=7,random_state=0)

# eclf1 = VotingClassifier(estimators=[
#          ('lr', clf1), ('rf', clf2), ('ada', clf3), ('dt',clf4),('knn',clf5),('gb',clf6)], voting='soft')
# eclf1 = eclf1.fit(X_train, Y_train)
# y_pred = eclf1.predict_proba(X_test)
# y_pred=y_pred[:,1]
# print(roc_auc_score(Y_test,y_pred))

# eclf2 = VotingClassifier(estimators=[
#         ('lr', clf1), ('rf', clf2), ('ada', clf3), ('dt',clf4),('knn',clf5),('gb',clf6)],voting='soft')
# eclf2 = eclf2.fit(X_train, Y_train)
# y_pred = eclf2.predict_proba(X_test)
# y_pred=y_pred[:,1]
# print(roc_auc_score(Y_test,y_pred))
eclf3 = VotingClassifier(estimators=[
       ('lr', clf1), ('rf', clf2), ('ada', clf3), ('dt',clf4),('knn',clf5),('gb',clf6)],voting='soft', 
                         weights=[1,10,50,2,2,90],flatten_transform=True)
eclf3 = eclf3.fit(X_train, Y_train)
y_pred = eclf3.predict_proba(X_test)
y_pred=y_pred[:,1]
print(roc_auc_score(Y_test,y_pred))

print(eclf3.transform(X_train).shape)

0.9237184163584927
(15000, 12)


In [142]:
eclf3.fit(X_traink,Y_traink)
y_pred = eclf3.predict_proba(kaggleTest)
y_pred=y_pred[:,1]

# Now output a file with the predictions on test data:
np.savetxt('MV2.csv',
  np.vstack((np.arange(len(y_pred)), y_pred)).T,
  '%d, %f',header='ID,Prob1',comments='',delimiter=',');

In [24]:
print(y_pred.shape)

(10000,)


In [117]:
gb= GradientBoostingClassifier(n_estimators=40, learning_rate=0.8,max_depth=2, min_samples_leaf=7,random_state=0)
gb= gb.fit(X_train,Y_train)
y_pred= gb.predict_proba(X_test)
y_pred=y_pred[:,1]
print(y_pred.shape)
print(y_pred)
print(Y_test.shape)
print(roc_auc_score(Y_test,y_pred))
gb.fit(X_traink,Y_traink)
y_pred =gb.predict_proba(kaggleTest)
y_pred=y_pred[:,1]
# Now output a file with the predictions on test data:
np.savetxt('xgb.csv',
  np.vstack((np.arange(len(y_pred)), y_pred)).T,
  '%d, %f',header='ID,Prob1',comments='',delimiter=',');


(6000,)
[0.91182746 0.35017698 0.28096627 ... 0.54036533 0.00493418 0.00191273]
(6000, 1)
0.9196122037958419


# Grid search cv

In [ ]:
from xgboost.sklearn import XGBClassifier
param = {
 'n_estimators':[50,100,150,200,300,400,500,600,800,850,900],
 'max_depth':[2,3,4,5,6,7,8],
 'min_child_weight':[2,3,4,5],
 'colsample_bytree':[0.2,0.6,0.8],
 'colsample_bylevel':[0.2,0.6,0.8]
}

gsearch1 = GridSearchCV(estimator = XGBClassifier( 
        objective= "multi:softmax", 
        num_class = 2,
        seed=1), 
    param_grid = param, 
    scoring="roc_auc",
    cv=10,
    verbose = 1)

# http://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
# mean_squared_error alternative.

gsearch1.fit(X_train, Y_train)
gsearch1.bestscore
gsearch1.bestparams
y_pred = gsearch1.predict_proba(X_test)
y_pred=y_pred[:,1]
print(roc_auc_score(Y_test,y_pred))
